In [1]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

c:\Users\Asus\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:

test_text = """I don't get it. I just don't. This is one of the most revered movies of all time. And I just hate it. I don't understand how anybody who isn't a child could like this.\n\nMy chief complaint is that everybody in the Land of Oz (including Dorothy) is extremely annoying. I spend the entire time just wanting everybody to calm down. The worst ones are the munchkins who speak in incredibly irritating voices, wear hideous costumes and sing a song that lasts a million years. Not to mention that they feel quite offensive towards little people to me. And nobody can talk properly either! (Somebody could argue that everyone in Oz is imagined by a child so that's why they behave so weirdly. That doesn't stop it from being extremely unpleasant to watch them, though.)\n\nThe music is also all terrible (except for Somewhere Over the Rainbow). Whenever a musical number started I would roll my eyes and pray that it wouldn't go on too long.\n\nI don't think this opinion could convince anyone who likes this movie because I'm basically just ranting, saying that I find it annoying. But that's how I feel. Everything just irritates and infuriates me. And I don't just hate old movies by the way: I love plenty of movies that are as old or older than The Wizard of Oz. I even like some children's movies from the time like Snow White and the Seven Dwarfs. So I don't know what happened here.\n\nIf I have to give some positives, I guess that the movie looks good for 1939 and the black-and-white section at the beginning is fine. (The scene with the fortune teller is actually kind of good.) And admittedly there are some pretty clever ideas like the transition from black-and-white to colour and the Wizard actually being a normal man. This movie is technically not a complete wash but I find it outrageously irritating.\n\nP. S.: Dorothy looks 17 but acts like she's 8 (she's meant to be 12) and I find this really unsettling."""

tokenizer = BertTokenizer.from_pretrained("saved_model_try_5")
model = BertForSequenceClassification.from_pretrained("saved_model_try_5")

In [5]:
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

inputs = tokenizer(test_text, padding=True, truncation=True, return_tensors="pt")
inputs = inputs.to(device)

with torch.no_grad():
    outputs = model(**inputs)

logits = outputs.logits
probabilities = torch.sigmoid(logits)  # Apply sigmoid to convert logits to probabilities
probabilities = probabilities.squeeze(0)

threshold = 0.5
predicted_labels = (probabilities > threshold).long()

probabilities = probabilities.tolist()
m_result = []
m_result.append(probabilities)

emotions = ["anger", "anticipation", "disgust", "fear", "joy", "love", "optimism", "pessimism", "sadness", "surprise", "trust"]

results = pd.DataFrame(m_result, columns=emotions)

results

,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,0.852917,0.097523,0.842072,0.112649,0.082671,0.031457,0.059509,0.202234,0.403145,0.145829,0.044241
